In [2]:
from rake_nltk import Rake
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import string
warnings.filterwarnings("ignore")

In [3]:
df_csv = pd.read_csv('C:/Users/camil/Desktop/Projeto Integrador/dados/final.csv')

In [5]:
df_csv.head()

,Unnamed: 0,tconst,primaryTitle,startYear,runtimeMinutes,genre_1,genre_2,genre_3,Item,Mês,...,directors_2_films,directors_1_ratingAverage,directors_2_ratingAverage,writer_1_films,writer_2_films,writer_1_ratingAverage,writer_2_ratingAverage,AvgRating,numFilmes,num_crew
0,66398,tt0118589,Glitter,2001,104,Drama,Music,Romance,3947,Setembro,...,0,6.90,NaN,0,1,NaN,6.900000,5.800000,0.25,10
1,67605,tt0120681,From Hell,2001,122,Horror,Mystery,Thriller,1732,Outubro,...,3,7.31,7.31,0,0,NaN,NaN,6.258212,1.00,10
2,67642,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,178,Action,Adventure,Drama,356,Dezembro,...,0,7.16,NaN,0,4,NaN,7.262979,6.533333,0.75,10
3,67655,tt0120755,Mission: Impossible II,2000,123,Action,Adventure,Thriller,297,Maio,...,0,7.11,NaN,3,0,7.099574,NaN,NaN,0.00,10
4,68298,tt0122459,Return to Me,2000,115,Comedy,Drama,Romance,2404,Abril,...,0,NaN,NaN,0,0,NaN,NaN,NaN,0.00,10


In [4]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 67 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            1490 non-null   int64  
 1   tconst                                1490 non-null   object 
 2   primaryTitle                          1490 non-null   object 
 3   startYear                             1490 non-null   int64  
 4   runtimeMinutes                        1490 non-null   int64  
 5   genre_1                               1490 non-null   object 
 6   genre_2                               1373 non-null   object 
 7   genre_3                               997 non-null    object 
 8   Item                                  1490 non-null   int64  
 9   Mês                                   1490 non-null   object 
 10  month                                 1490 non-null   float64
 11  day              

In [8]:
df_csv.columns

Index(['Unnamed: 0', 'tconst', 'primaryTitle', 'startYear', 'runtimeMinutes',
       'genre_1', 'genre_2', 'genre_3', 'Item', 'Mês', 'month', 'day', 'year',
       'Data Portugues', 'FW', 'Worldwide_Gross', 'Production_Budget',
       'rentability', 'log_rentability', 'rating', 'movie_info',
       'on_streaming_date', 'studio_name', 'tomatometer_status',
       'tomatometer_rating', 'tomatometer_count', 'audience_status',
       'audience_want_to_see_count', 'audience_rating', 'audience_count',
       'audience_top_critics_count', 'audience_fresh_critics_count',
       'audience_rotten_critics_count', 'audience_fresh_top_critics_count',
       'audience_rotten_rotten_critics_count', 'tomatometer_avg_rating',
       'audience_top_critics_avg_rating', 'score_sentiment',
       'audience_avg_rating', 'Negative', 'Positive', 'All', 'negative_perc',
       'positive_perc', 'NPremium_Indicado', 'NPremium_Win',
       'Premium_Indicado', 'Premium_Win', 'movieyear', 'averageRating',
       'n

In [9]:
# criar copias dos dois datasets para manipular os dados
df = df_csv.copy()
#df2 = dfx.copy()

# excluir linhas repetidas
df.drop_duplicates(subset = 'movie_info', inplace = True)

# selecionar colunas de interesse dos dois datasets
df = df[['primaryTitle', 'movie_info']]
#df2 = df_eda[['primaryTitle','movie_name_imdb']]

# excluir missing values
df.dropna(inplace=True)

# juntar os dois datasets
#df.append(df2, ignore_index=True)

In [10]:
# remoção da pontuação de 'Plot'
df['movie_info'] = df['movie_info'].str.replace('[{}]'.format(string.punctuation), '')

In [5]:
 # inicialização das lista que será a nova coluna
df['Key_words'] = ''  

# instanciação do objeto do Rake para descartar as 'stop words' (baseado naquilo que está no NLTK)
r = Rake()

for index, row in df.iterrows():
    # extração das key words - que já remove as 'stop words' do texto
    r.extract_keywords_from_text(row['movie_info'])   
    # dicionário com as key words e scores
    key_words_dict_scores = r.get_word_degrees()    
    # nova coluna que será inserida
    row['Key_words'] = list(key_words_dict_scores.keys())   

In [6]:
'''# to extract all genre into a list, only the first three actors into a list, and all directors into a list
df['genres'] = df['genres'].map(lambda x: x.split(','))
df['directors'] = df['directors'].map(lambda x: x.split(','))
df['casts'] = df['casts'].map(lambda x: x.split(',')[:3])'''

In [7]:
'''# Retirada dos espaços dos campos e conversão para minúsculas
for index, row in df.iterrows():
    row['genres'] = [x.lower().replace(' ','') for x in row['genres']] 
    row['casts'] = [x.lower().replace(' ','') for x in row['casts']]
    row['directors'] = [x.lower().replace(' ','') for x in row['directors']]'''

In [8]:
# inicialização da lista
df['Bag_of_words'] = ''
# colunas que farão parte da Bag of Words
#columns = ['genres', 'directors', 'casts', 'Key_words']
columns = [ 'Key_words']
# percorre cada coluna e adiciona cada palavra individualmente
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words

In [9]:
# remoção dos "espaços" de todas as palavras na BoW
df['Bag_of_words'] = df['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')

In [36]:
len(df)

1263

In [37]:
len(df_bag)

1263

In [11]:
df_bag = df[['movie_name_imdb','Bag_of_words']]

In [12]:
# Geração da Matriz com a frequência de cada palavra com filme 250 x quantidade total de palavras únicas
count = CountVectorizer()
count_matrix = count.fit_transform(df_bag['Bag_of_words'])

In [13]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [41]:
df_bag.reset_index(inplace = True)

In [42]:
# to create a Series for movie titles which can be used as indices (each index is mapped to a movie title)
indices = pd.Series(df_bag['movie_name_imdb'])

In [39]:
idx = indices[indices == 'Liz and the Blue Bird'].index[0] 

In [43]:
def recommend(title, cosine_sim = cosine_sim):
    # inicialização da lista
    recommended_movies = []
    # captura o índice que corresponde ao título
    idx = indices[indices == title].index[0]   
    # scores de similaridade em ordem descendente
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   
    # captura os 10 filmes com o maior índice de similaridade
    top_10_indices = list(score_series.iloc[1:11].index)   
    # [1:11] para excluir o primeiro (0) que é o próprio filme selecionado
    
    # monta uma lista com os filmes retornados
    for i in top_10_indices:   
        recommended_movies.append(list(df_bag['movie_name_imdb'])[i])
        
    return recommended_movies

In [44]:
df_bag.movie_name_imdb.unique()

array(['The Notebook', 'Miracle', 'Elf', ..., 'Bullitt County',
       'Liz and the Blue Bird', 'Eve'], dtype=object)

In [45]:
filme = recommend("Liz and the Blue Bird")

# recomendacao para o filme que sera produzido
#filme = recommend(df2.movie_name)

In [56]:
filme

['The Miracle Season',
 'Tape',
 'Peter and Vandy',
 'Forever My Girl',
 'The English Teacher',
 'Miss You Already',
 'Annapolis',
 'Lost and Delirious',
 'Dumb and Dumberer: When Harry Met Lloyd',
 'Cloaca']

In [ ]:
#imdb
#analise de sentimento
# negative_perc
# positive_perc
#nota total e n de votos
#numvotos
#AvgRating

In [57]:
df_rec = df_csv[df_csv.movie_name_imdb.isin(filme)]

In [18]:
print(float(df_csv.tomatometer_avg_rating[df_csv.movie_name_imdb.isin(filme)].mean()))
print(float(df_csv.tomatometer_avg_rating[df_csv.movie_name_imdb.isin(filme)].min()))
print(float(df_csv.tomatometer_avg_rating[df_csv.movie_name_imdb.isin(filme)].max()))

# pegar a rating_avg desse filme mais proximo e salvar como uma nova coluna para o filme a ser feito
#df_x['rating_avg'] = float(df_csv.tomatometer_avg_rating[df_csv.movie_name_imdb.isin(filme)].mean())
#df_x['rating_min'] = float(df_csv.tomatometer_avg_rating[df_csv.movie_name_imdb.isin(filme)].min())
#df_x['rating_max'] = float(df_csv.tomatometer_avg_rating[df_csv.movie_name_imdb.isin(filme)].max())

5.373870967741938
0.0
6.64
